This notebook generates XML files for each image in the dataset that contain a list of objects

In [3]:
import os
import dicttoxml
from xml.dom.minidom import parseString
from collections import OrderedDict
import numpy as np

import helper # this module contains some function that are used by a few of the notebooks on this matter

N_CLASSES = 10
BAND = 'RGB' # which band to use
MASK_SIZES = {'A': 128, 'M': 800, 'P': 3*1024, 'RGB': 1024} # size of the Mask ARRAY to use
DEPTHS = {'A': 8, 'M': 8, 'P': 1, 'RGB': 3}
data_dir = '/data/dstl'
if BAND == 'RGB':
    dir_name = os.path.join(data_dir, 'three_band')
else:
    dir_name =  os.path.join(data_dir, 'sixteen_band')
MIN_BBOX_AREA = 10 # we need to avoid zero width or zero heigth bboxes, but also small bboxes are likely underisable
class_names = {0: 'buildings', 
               1: 'misc. manmade structures',
               2: 'road',
               3: 'track',
               4: 'trees',
               5: 'crops',
               6: 'waterway',
               7: 'standing water',
               8: 'vehicle large',
               9: 'vehicle small'}

In [4]:
get = np.load(os.path.join(data_dir, 'bboxes_{}.npz'.format(BAND)))
in_bboxes = get['bboxes'][()]
im_sizes = get['im_sizes'][()]

Before looping over all image ids, we will define the functions needed to be performed for each image and run them on an arbitrary `im_id` defined below. 

In [8]:
im_id = '6010_1_2' # an arbitrary image that contains at least a few bboxes

The cell below defines a function that contains two xml snippets, one for the image information (`xml_header`) and one for the objects in this image (`xml_objects`). This split is convenient because `xml_header` is based on a _named dict_, whereas `xml_objects` is based on an (unnnamed) _list_.

In [23]:
item_name_func = lambda parent: 'object'

def create_xml_snippets(im_id):
    out_dict = OrderedDict()
    out_dict['folder'] = dir_name.split('/')[-1]
    out_dict['filename'] = im_id+'.tiff'
    out_dict['source'] = {'database': 'dstl Kaggle case', 'annotations': 'dstl Kaggle case'}
    out_dict['owner'] = 'dstl'
    out_dict['source'] = {'width': im_sizes[im_id][1], 'height': im_sizes[im_id][0], 'depth': DEPTHS[BAND]}
    out_dict['segmented'] = 0
    
    xml_header = dicttoxml.dicttoxml(out_dict, attr_type=False, root=False)
    
    object_list = []
    # add objects (if any!)
    object_dict = OrderedDict()
    if im_id in in_bboxes:
        for c in range(N_CLASSES):
            for bbox in in_bboxes[im_id][c]:
                if bbox['w'] * bbox['h'] > MIN_BBOX_AREA:
                    object_dict['name'] = class_names[c]
                    object_dict['pose'] = ''
                    object_dict['truncated'] = 0
                    object_dict['difficult'] = 0
                    object_dict['bndbox'] = {'xmin': bbox['x0'], 'ymin': bbox['y0'], 'xmax': bbox['x1'], 'ymax': bbox['y1']}
                    object_list.append(object_dict)
    
    xml_objects = dicttoxml.dicttoxml(object_list, item_func=item_name_func, attr_type=False, root=False)
    return xml_header, xml_objects
    
xml_header, xml_objects = create_xml_snippets(im_id)
    

In [24]:
c = 1 # class for which im_id contains at least one object
print in_bboxes[im_id][c][0] # the first bbox of class c in in image im_id

{'h': 8, 'y1': 3182, 'w': 10, 'x': 3310, 'y': 3178, 'y0': 3174, 'x0': 3305, 'x1': 3315}


In [25]:
print xml_header
print xml_objects

<folder>three_band</folder><filename>6010_1_2.tiff</filename><source><width>3396</width><depth>3</depth><height>3349</height></source><owner>dstl</owner><segmented>0</segmented>
<object><name>trees</name><pose></pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>178</xmin><ymin>1486</ymin><ymax>1679</ymax><xmax>309</xmax></bndbox></object><object><name>trees</name><pose></pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>178</xmin><ymin>1486</ymin><ymax>1679</ymax><xmax>309</xmax></bndbox></object><object><name>trees</name><pose></pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>178</xmin><ymin>1486</ymin><ymax>1679</ymax><xmax>309</xmax></bndbox></object><object><name>trees</name><pose></pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>178</xmin><ymin>1486</ymin><ymax>1679</ymax><xmax>309</xmax></bndbox></object><object><name>trees</name><pose></pose><truncated>0</truncated><difficult>0</difficult><bndbox><xmin>178</

In [26]:
def concatenate_xml_snippets(xml_header, xml_objects):
    return '<annotation>\n'+xml_header+'\n'+xml_objects+'\n</annotation>'
xml = concatenate_xml_snippets(xml_header, xml_objects)
dom = parseString(xml)
print dom.toprettyxml()

<?xml version="1.0" ?>
<annotation>
	

	<folder>three_band</folder>
	<filename>6010_1_2.tiff</filename>
	<source>
		<width>3396</width>
		<depth>3</depth>
		<height>3349</height>
	</source>
	<owner>dstl</owner>
	<segmented>0</segmented>
	

	<object>
		<name>trees</name>
		<pose/>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>178</xmin>
			<ymin>1486</ymin>
			<ymax>1679</ymax>
			<xmax>309</xmax>
		</bndbox>
	</object>
	<object>
		<name>trees</name>
		<pose/>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>178</xmin>
			<ymin>1486</ymin>
			<ymax>1679</ymax>
			<xmax>309</xmax>
		</bndbox>
	</object>
	<object>
		<name>trees</name>
		<pose/>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>178</xmin>
			<ymin>1486</ymin>
			<ymax>1679</ymax>
			<xmax>309</xmax>
		</bndbox>
	</object>
	<object>
		<name>trees</name>
		<pose/>
		<truncated>0</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>178</xmin>
	

In [27]:
def write_xml(xml, im_id):
    with open(os.path.join(dir_name, 'Annotations', im_id+'_RGB.xml'), 'w') as f:
        dom = parseString(xml)
        f.write(dom.toprettyxml())

write_xml(xml, im_id)

With each of the functions above defined, now a loop over all the images creates a `.xml` file for each image.

In [28]:
for im_id in helper.image_iterator():
    xml_header, xml_objects = create_xml_snippets(im_id)
    xml = concatenate_xml_snippets(xml_header, xml_objects)
    write_xml(xml, im_id)
    